# Feature Engineering

In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)

fighters = pd.read_csv("../data/fighter_stats.csv")
bouts = pd.read_csv("../data/bout_stats.csv")
relevant = ["UID", "Wins", "Losses", "Draws", "NC", "Height", "Reach", "Stance", "DOB"]
fstats_R = fighters[relevant].rename(columns={"UID": "R_UID",
                                              "Wins": "R_Wins Overall",
                                              "Losses": "R_Losses Overall",
                                              "Draws": "R_Draws Overall",
                                              "NC": "R_NC Overall",
                                              "Height": "R_Height",
                                              "Reach": "R_Reach",
                                              "Stance": "R_Stance",
                                              "DOB": "R_DOB"})
fstats_B = fighters[relevant].rename(columns={"UID": "B_UID",
                                              "Wins": "B_Wins Overall",
                                              "Losses": "B_Losses Overall",
                                              "Draws": "B_Draws Overall",
                                              "NC": "B_NC Overall",
                                              "Height": "B_Height",
                                              "Reach": "B_Reach",
                                              "Stance": "B_Stance",
                                              "DOB": "B_DOB"})
bouts = bouts.merge(fstats_R, on="R_UID", how="left")
bouts = bouts.merge(fstats_B, on="B_UID", how="left")
bouts = bouts.drop(["URL", "R_Name", "B_Name"], axis=1)

In [2]:
bouts

,Event,Date,Location,R_UID,B_UID,R_Result,B_Result,Bout Type,Method,Round,Time,Format,Total Time,R_KD,B_KD,R_Total Str. Landed,R_Total Str. Attempted,B_Total Str. Landed,B_Total Str. Attempted,R_TD Landed,R_TD Attempted,B_TD Landed,B_TD Attempted,R_TD %,B_TD %,R_Sub. Att,B_Sub. Att,R_Rev.,B_Rev.,R_Ctrl,B_Ctrl,R_KD_R1,B_KD_R1,R_Total Str. Landed_R1,R_Total Str. Attempted_R1,B_Total Str. Landed_R1,B_Total Str. Attempted_R1,R_TD Landed_R1,R_TD Attempted_R1,B_TD Landed_R1,B_TD Attempted_R1,R_TD %_R1,B_TD %_R1,R_Sub. Att_R1,B_Sub. Att_R1,R_Rev._R1,B_Rev._R1,R_Ctrl_R1,B_Ctrl_R1,R_KD_R2,B_KD_R2,R_Total Str. Landed_R2,R_Total Str. Attempted_R2,B_Total Str. Landed_R2,B_Total Str. Attempted_R2,R_TD Landed_R2,R_TD Attempted_R2,B_TD Landed_R2,B_TD Attempted_R2,R_TD %_R2,B_TD %_R2,R_Sub. Att_R2,B_Sub. Att_R2,R_Rev._R2,B_Rev._R2,R_Ctrl_R2,B_Ctrl_R2,R_KD_R3,B_KD_R3,R_Total Str. Landed_R3,R_Total Str. Attempted_R3,B_Total Str. Landed_R3,B_Total Str. Attempted_R3,R_TD Landed_R3,R_TD Attempted_R3,B_TD Landed_R3,B_TD Attempted_R3,R_TD %_R3,B_TD %_R3,R_Sub. Att_R3,B_Sub. Att_R3,R_Rev._R3,B_Rev._R3,R_Ctrl_R3,B_Ctrl_R3,R_KD_R4,B_KD_R4,R_Total Str. Landed_R4,R_Total Str. Attempted_R4,B_Total Str. Landed_R4,B_Total Str. Attempted_R4,R_TD Landed_R4,R_TD Attempted_R4,B_TD Landed_R4,B_TD Attempted_R4,R_TD %_R4,B_TD %_R4,R_Sub. Att_R4,B_Sub. Att_R4,R_Rev._R4,B_Rev._R4,R_Ctrl_R4,B_Ctrl_R4,R_KD_R5,B_KD_R5,R_Total Str. Landed_R5,R_Total Str. Attempted_R5,B_Total Str. Landed_R5,B_Total Str. Attempted_R5,R_TD Landed_R5,R_TD Attempted_R5,B_TD Landed_R5,B_TD Attempted_R5,R_TD %_R5,B_TD %_R5,R_Sub. Att_R5,B_Sub. Att_R5,R_Rev._R5,B_Rev._R5,R_Ctrl_R5,B_Ctrl_R5,R_KD_R6,B_KD_R6,R_Total Str. Landed_R6,R_Total Str. Attempted_R6,B_Total Str. Landed_R6,B_Total Str. Attempted_R6,R_TD Landed_R6,R_TD Attempted_R6,B_TD Landed_R6,B_TD Attempted_R6,R_TD %_R6,B_TD %_R6,R_Sub. Att_R6,B_Sub. Att_R6,R_Rev._R6,B_Rev._R6,R_Ctrl_R6,B_Ctrl_R6,R_Sig. Str. Landed,R_Sig. Str. Attempted,B_Sig. Str. Landed,B_Sig. Str. Attempted,R_Sig. Str. %,B_Sig. Str. %,R_Head Landed,R_Head Attempted,B_Head Landed,B_Head Attempted,R_Body Landed,R_Body Attempted,B_Body Landed,B_Body Attempted,R_Leg Landed,R_Leg Attempted,B_Leg Landed,B_Leg Attempted,R_Distance Landed,R_Distance Attempted,B_Distance Landed,B_Distance Attempted,R_Clinch Landed,R_Clinch Attempted,B_Clinch Landed,B_Clinch Attempted,R_Ground Landed,R_Ground Attempted,B_Ground Landed,B_Ground Attempted,R_Sig. Str. Landed_R1,R_Sig. Str. Attempted_R1,B_Sig. Str. Landed_R1,B_Sig. Str. Attempted_R1,R_Sig. Str. %_R1,B_Sig. Str. %_R1,R_Head Landed_R1,R_Head Attempted_R1,B_Head Landed_R1,B_Head Attempted_R1,R_Body Landed_R1,R_Body Attempted_R1,B_Body Landed_R1,B_Body Attempted_R1,R_Leg Landed_R1,R_Leg Attempted_R1,B_Leg Landed_R1,B_Leg Attempted_R1,R_Distance Landed_R1,R_Distance Attempted_R1,B_Distance Landed_R1,B_Distance Attempted_R1,R_Clinch Landed_R1,R_Clinch Attempted_R1,B_Clinch Landed_R1,B_Clinch Attempted_R1,R_Ground Landed_R1,R_Ground Attempted_R1,B_Ground Landed_R1,B_Ground Attempted_R1,R_Sig. Str. Landed_R2,R_Sig. Str. Attempted_R2,B_Sig. Str. Landed_R2,B_Sig. Str. Attempted_R2,R_Sig. Str. %_R2,B_Sig. Str. %_R2,R_Head Landed_R2,R_Head Attempted_R2,B_Head Landed_R2,B_Head Attempted_R2,R_Body Landed_R2,R_Body Attempted_R2,B_Body Landed_R2,B_Body Attempted_R2,R_Leg Landed_R2,R_Leg Attempted_R2,B_Leg Landed_R2,B_Leg Attempted_R2,R_Distance Landed_R2,R_Distance Attempted_R2,B_Distance Landed_R2,B_Distance Attempted_R2,R_Clinch Landed_R2,R_Clinch Attempted_R2,B_Clinch Landed_R2,B_Clinch Attempted_R2,R_Ground Landed_R2,R_Ground Attempted_R2,B_Ground Landed_R2,B_Ground Attempted_R2,R_Sig. Str. Landed_R3,R_Sig. Str. Attempted_R3,B_Sig. Str. Landed_R3,B_Sig. Str. Attempted_R3,R_Sig. Str. %_R3,B_Sig. Str. %_R3,R_Head Landed_R3,R_Head Attempted_R3,B_Head Landed_R3,B_Head Attempted_R3,R_Body Landed_R3,R_Body Attempted_R3,B_Body Landed_R3,B_Body Attempted_R3,R_Leg Landed_R3,R_Leg Attempted_R3,B_Leg Landed_R3,B_Leg Attempted_R3,R_Distance Landed_R3,R_Di

## Create dataframes to be used to generate features

### All bouts

In [3]:
temp = bouts.copy().rename_axis("TEMP").reset_index()
common = ["TEMP", "Event", "Date", "Location", "Bout Type", "Method", "Round", "Time", "Format", "Total Time"]
red_cols = [col for col in temp.columns if col.startswith("R_")]
blue_cols = [col for col in temp.columns if col.startswith("B_")]

reds = temp[common + red_cols + blue_cols]
reds = reds.rename(columns=lambda x: x.replace("R_", "F_") if x.startswith("R_") else x.replace("B_", "O_") if x.startswith("B_") else x)
reds["TEMP2"] = 1
blues = temp[common + blue_cols + red_cols]
blues = blues.rename(columns=lambda x: x.replace("B_", "F_") if x.startswith("B_") else x.replace("R_", "O_") if x.startswith("R_") else x)
blues["TEMP2"] = 2

feature_df = pd.concat([reds, blues]).sort_values(["TEMP", "TEMP2"]).drop(["TEMP", "TEMP2"], axis=1)
feature_df["Date"] = pd.to_datetime(feature_df["Date"])
feature_df

,Event,Date,Location,Bout Type,Method,Round,Time,Format,Total Time,F_UID,F_Result,F_KD,F_Total Str. Landed,F_Total Str. Attempted,F_TD Landed,F_TD Attempted,F_TD %,F_Sub. Att,F_Rev.,F_Ctrl,F_KD_R1,F_Total Str. Landed_R1,F_Total Str. Attempted_R1,F_TD Landed_R1,F_TD Attempted_R1,F_TD %_R1,F_Sub. Att_R1,F_Rev._R1,F_Ctrl_R1,F_KD_R2,F_Total Str. Landed_R2,F_Total Str. Attempted_R2,F_TD Landed_R2,F_TD Attempted_R2,F_TD %_R2,F_Sub. Att_R2,F_Rev._R2,F_Ctrl_R2,F_KD_R3,F_Total Str. Landed_R3,F_Total Str. Attempted_R3,F_TD Landed_R3,F_TD Attempted_R3,F_TD %_R3,F_Sub. Att_R3,F_Rev._R3,F_Ctrl_R3,F_KD_R4,F_Total Str. Landed_R4,F_Total Str. Attempted_R4,F_TD Landed_R4,F_TD Attempted_R4,F_TD %_R4,F_Sub. Att_R4,F_Rev._R4,F_Ctrl_R4,F_KD_R5,F_Total Str. Landed_R5,F_Total Str. Attempted_R5,F_TD Landed_R5,F_TD Attempted_R5,F_TD %_R5,F_Sub. Att_R5,F_Rev._R5,F_Ctrl_R5,F_KD_R6,F_Total Str. Landed_R6,F_Total Str. Attempted_R6,F_TD Landed_R6,F_TD Attempted_R6,F_TD %_R6,F_Sub. Att_R6,F_Rev._R6,F_Ctrl_R6,F_Sig. Str. Landed,F_Sig. Str. Attempted,F_Sig. Str. %,F_Head Landed,F_Head Attempted,F_Body Landed,F_Body Attempted,F_Leg Landed,F_Leg Attempted,F_Distance Landed,F_Distance Attempted,F_Clinch Landed,F_Clinch Attempted,F_Ground Landed,F_Ground Attempted,F_Sig. Str. Landed_R1,F_Sig. Str. Attempted_R1,F_Sig. Str. %_R1,F_Head Landed_R1,F_Head Attempted_R1,F_Body Landed_R1,F_Body Attempted_R1,F_Leg Landed_R1,F_Leg Attempted_R1,F_Distance Landed_R1,F_Distance Attempted_R1,F_Clinch Landed_R1,F_Clinch Attempted_R1,F_Ground Landed_R1,F_Ground Attempted_R1,F_Sig. Str. Landed_R2,F_Sig. Str. Attempted_R2,F_Sig. Str. %_R2,F_Head Landed_R2,F_Head Attempted_R2,F_Body Landed_R2,F_Body Attempted_R2,F_Leg Landed_R2,F_Leg Attempted_R2,F_Distance Landed_R2,F_Distance Attempted_R2,F_Clinch Landed_R2,F_Clinch Attempted_R2,F_Ground Landed_R2,F_Ground Attempted_R2,F_Sig. Str. Landed_R3,F_Sig. Str. Attempted_R3,F_Sig. Str. %_R3,F_Head Landed_R3,F_Head Attempted_R3,F_Body Landed_R3,F_Body Attempted_R3,F_Leg Landed_R3,F_Leg Attempted_R3,F_Distance Landed_R3,F_Distance Attempted_R3,F_Clinch Landed_R3,F_Clinch Attempted_R3,F_Ground Landed_R3,F_Ground Attempted_R3,F_Sig. Str. Landed_R4,F_Sig. Str. Attempted_R4,F_Sig. Str. %_R4,F_Head Landed_R4,F_Head Attempted_R4,F_Body Landed_R4,F_Body Attempted_R4,F_Leg Landed_R4,F_Leg Attempted_R4,F_Distance Landed_R4,F_Distance Attempted_R4,F_Clinch Landed_R4,F_Clinch Attempted_R4,F_Ground Landed_R4,F_Ground Attempted_R4,F_Sig. Str. Landed_R5,F_Sig. Str. Attempted_R5,F_Sig. Str. %_R5,F_Head Landed_R5,F_Head Attempted_R5,F_Body Landed_R5,F_Body Attempted_R5,F_Leg Landed_R5,F_Leg Attempted_R5,F_Distance Landed_R5,F_Distance Attempted_R5,F_Clinch Landed_R5,F_Clinch Attempted_R5,F_Ground Landed_R5,F_Ground Attempted_R5,F_Sig. Str. Landed_R6,F_Sig. Str. Attempted_R6,F_Sig. Str. %_R6,F_Head Landed_R6,F_Head Attempted_R6,F_Body Landed_R6,F_Body Attempted_R6,F_Leg Landed_R6,F_Leg Attempted_R6,F_Distance Landed_R6,F_Distance Attempted_R6,F_Clinch Landed_R6,F_Clinch Attempted_R6,F_Ground Landed_R6,F_Ground Attempted_R6,F_Wins Overall,F_Losses Overall,F_Draws Overall,F_NC Overall,F_Height,F_Reach,F_Stance,F_DOB,O_UID,O_Result,O_KD,O_Total Str. Landed,O_Total Str. Attempted,O_TD Landed,O_TD Attempted,O_TD %,O_Sub. Att,O_Rev.,O_Ctrl,O_KD_R1,O_Total Str. Landed_R1,O_Total Str. Attempted_R1,O_TD Landed_R1,O_TD Attempted_R1,O_TD %_R1,O_Sub. Att_R1,O_Rev._R1,O_Ctrl_R1,O_KD_R2,O_Total Str. Landed_R2,O_Total Str. Attempted_R2,O_TD Landed_R2,O_TD Attempted_R2,O_TD %_R2,O_Sub. Att_R2,O_Rev._R2,O_Ctrl_R2,O_KD_R3,O_Total Str. Landed_R3,O_Total Str. Attempted_R3,O_TD Landed_R3,O_TD Attempted_R3,O_TD %_R3,O_Sub. Att_R3,O_Rev._R3,O_Ctrl_R3,O_KD_R4,O_Total Str. Landed_R4,O_Total Str. Attempted_R4,O_TD Landed_R4,O_TD Attempted_R4,O_TD %_R4,O_Sub. Att_R4,O_Rev._R4,O_Ctrl_R4,O_KD_R5,O_Total Str. Landed_R5,O_Total Str. Attempted_R5,O_TD Landed_R5,O_TD Attempted_R5,O_TD %_R5,O_Sub. Att_R5,O_Rev._R5,O_Ctrl_R5,O_KD_R6,O_Total Str. Landed_R6,O_Total Str. Attempted_R6,O_TD Lande

### UFC bouts only

In [4]:
def is_ufc_event(event):
    valid_list = ["UFC: ", "UFC ", "The Ultimate Fighter", "Ortiz vs Shamrock 3: The Final Chapter"]
    if "Road to UFC" in event:
        return False
    elif any([valid in event for valid in valid_list]):
        return True
    return False

feature_df_ufc_only = feature_df.copy()
feature_df_ufc_only = feature_df_ufc_only[feature_df_ufc_only["Event"].apply(is_ufc_event)]
feature_df_ufc_only

,Event,Date,Location,Bout Type,Method,Round,Time,Format,Total Time,F_UID,F_Result,F_KD,F_Total Str. Landed,F_Total Str. Attempted,F_TD Landed,F_TD Attempted,F_TD %,F_Sub. Att,F_Rev.,F_Ctrl,F_KD_R1,F_Total Str. Landed_R1,F_Total Str. Attempted_R1,F_TD Landed_R1,F_TD Attempted_R1,F_TD %_R1,F_Sub. Att_R1,F_Rev._R1,F_Ctrl_R1,F_KD_R2,F_Total Str. Landed_R2,F_Total Str. Attempted_R2,F_TD Landed_R2,F_TD Attempted_R2,F_TD %_R2,F_Sub. Att_R2,F_Rev._R2,F_Ctrl_R2,F_KD_R3,F_Total Str. Landed_R3,F_Total Str. Attempted_R3,F_TD Landed_R3,F_TD Attempted_R3,F_TD %_R3,F_Sub. Att_R3,F_Rev._R3,F_Ctrl_R3,F_KD_R4,F_Total Str. Landed_R4,F_Total Str. Attempted_R4,F_TD Landed_R4,F_TD Attempted_R4,F_TD %_R4,F_Sub. Att_R4,F_Rev._R4,F_Ctrl_R4,F_KD_R5,F_Total Str. Landed_R5,F_Total Str. Attempted_R5,F_TD Landed_R5,F_TD Attempted_R5,F_TD %_R5,F_Sub. Att_R5,F_Rev._R5,F_Ctrl_R5,F_KD_R6,F_Total Str. Landed_R6,F_Total Str. Attempted_R6,F_TD Landed_R6,F_TD Attempted_R6,F_TD %_R6,F_Sub. Att_R6,F_Rev._R6,F_Ctrl_R6,F_Sig. Str. Landed,F_Sig. Str. Attempted,F_Sig. Str. %,F_Head Landed,F_Head Attempted,F_Body Landed,F_Body Attempted,F_Leg Landed,F_Leg Attempted,F_Distance Landed,F_Distance Attempted,F_Clinch Landed,F_Clinch Attempted,F_Ground Landed,F_Ground Attempted,F_Sig. Str. Landed_R1,F_Sig. Str. Attempted_R1,F_Sig. Str. %_R1,F_Head Landed_R1,F_Head Attempted_R1,F_Body Landed_R1,F_Body Attempted_R1,F_Leg Landed_R1,F_Leg Attempted_R1,F_Distance Landed_R1,F_Distance Attempted_R1,F_Clinch Landed_R1,F_Clinch Attempted_R1,F_Ground Landed_R1,F_Ground Attempted_R1,F_Sig. Str. Landed_R2,F_Sig. Str. Attempted_R2,F_Sig. Str. %_R2,F_Head Landed_R2,F_Head Attempted_R2,F_Body Landed_R2,F_Body Attempted_R2,F_Leg Landed_R2,F_Leg Attempted_R2,F_Distance Landed_R2,F_Distance Attempted_R2,F_Clinch Landed_R2,F_Clinch Attempted_R2,F_Ground Landed_R2,F_Ground Attempted_R2,F_Sig. Str. Landed_R3,F_Sig. Str. Attempted_R3,F_Sig. Str. %_R3,F_Head Landed_R3,F_Head Attempted_R3,F_Body Landed_R3,F_Body Attempted_R3,F_Leg Landed_R3,F_Leg Attempted_R3,F_Distance Landed_R3,F_Distance Attempted_R3,F_Clinch Landed_R3,F_Clinch Attempted_R3,F_Ground Landed_R3,F_Ground Attempted_R3,F_Sig. Str. Landed_R4,F_Sig. Str. Attempted_R4,F_Sig. Str. %_R4,F_Head Landed_R4,F_Head Attempted_R4,F_Body Landed_R4,F_Body Attempted_R4,F_Leg Landed_R4,F_Leg Attempted_R4,F_Distance Landed_R4,F_Distance Attempted_R4,F_Clinch Landed_R4,F_Clinch Attempted_R4,F_Ground Landed_R4,F_Ground Attempted_R4,F_Sig. Str. Landed_R5,F_Sig. Str. Attempted_R5,F_Sig. Str. %_R5,F_Head Landed_R5,F_Head Attempted_R5,F_Body Landed_R5,F_Body Attempted_R5,F_Leg Landed_R5,F_Leg Attempted_R5,F_Distance Landed_R5,F_Distance Attempted_R5,F_Clinch Landed_R5,F_Clinch Attempted_R5,F_Ground Landed_R5,F_Ground Attempted_R5,F_Sig. Str. Landed_R6,F_Sig. Str. Attempted_R6,F_Sig. Str. %_R6,F_Head Landed_R6,F_Head Attempted_R6,F_Body Landed_R6,F_Body Attempted_R6,F_Leg Landed_R6,F_Leg Attempted_R6,F_Distance Landed_R6,F_Distance Attempted_R6,F_Clinch Landed_R6,F_Clinch Attempted_R6,F_Ground Landed_R6,F_Ground Attempted_R6,F_Wins Overall,F_Losses Overall,F_Draws Overall,F_NC Overall,F_Height,F_Reach,F_Stance,F_DOB,O_UID,O_Result,O_KD,O_Total Str. Landed,O_Total Str. Attempted,O_TD Landed,O_TD Attempted,O_TD %,O_Sub. Att,O_Rev.,O_Ctrl,O_KD_R1,O_Total Str. Landed_R1,O_Total Str. Attempted_R1,O_TD Landed_R1,O_TD Attempted_R1,O_TD %_R1,O_Sub. Att_R1,O_Rev._R1,O_Ctrl_R1,O_KD_R2,O_Total Str. Landed_R2,O_Total Str. Attempted_R2,O_TD Landed_R2,O_TD Attempted_R2,O_TD %_R2,O_Sub. Att_R2,O_Rev._R2,O_Ctrl_R2,O_KD_R3,O_Total Str. Landed_R3,O_Total Str. Attempted_R3,O_TD Landed_R3,O_TD Attempted_R3,O_TD %_R3,O_Sub. Att_R3,O_Rev._R3,O_Ctrl_R3,O_KD_R4,O_Total Str. Landed_R4,O_Total Str. Attempted_R4,O_TD Landed_R4,O_TD Attempted_R4,O_TD %_R4,O_Sub. Att_R4,O_Rev._R4,O_Ctrl_R4,O_KD_R5,O_Total Str. Landed_R5,O_Total Str. Attempted_R5,O_TD Landed_R5,O_TD Attempted_R5,O_TD %_R5,O_Sub. Att_R5,O_Rev._R5,O_Ctrl_R5,O_KD_R6,O_Total Str. Landed_R6,O_Total Str. Attempted_R6,O_TD Lande